In [1]:
# %pip install beautifulsoup4
# %pip install requests

import urllib.robotparser
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup


## User Agent

In [2]:
# Common User Agent List
# https://www.networkinghowtos.com/howto/common-user-agent-list/
user_agent = "Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv 11.0) like Gecko"

## Selection of the data download site

In [ ]:
url = "https://www.ceneo.pl/"

#ID Samsung Galaxy S20 SM-G980 8/128GB Szary
tail = "91074688"

## Checking the rules for downloading data

In [3]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url(url + "robots.txt")
rp.read()

In [5]:
headers = requests.utils.default_headers()

headers.update(
    {
        "User-Agent": user_agent,
    }
)

In [7]:
store = []
rating = []
title = []
price = []
rating_number = []

## Selecting data

In [8]:
if rp.can_fetch(user_agent, url+tail):
    response = requests.get(url+tail, headers=headers)

    if response.ok:
        page = BeautifulSoup(response.text, 'html.parser')

        # Product price
        temp_value = page.select('ul > li > div > div.product-offer__container.js_product-offer > div.product-offer__store-with-product > div.product-offer__product.js_product-offer__product.js_productName.specific-variant-content > div.product-offer__product__price > a > span > span > span.value')
        temp_penny = page.select('ul > li > div > div.product-offer__container.js_product-offer > div.product-offer__store-with-product > div.product-offer__product.js_product-offer__product.js_productName.specific-variant-content > div.product-offer__product__price > a > span > span > span.penny')
        for i in range(len(temp_value)):
            price.append(temp_value[i].text+temp_penny[i].text)
        price = [w.replace(' ', '') for w in price]

        # Store name
        temp_name = page.select('div.product-offer a.js_product-offer-link')
        for i in range(len(temp_name)):
            store.append(temp_name[i].text)
        store = [w.replace('Dane i opinie o ', '') for w in store]

        # Store rating
        temp_rating = page.select('ul > li > div > div.product-offer__container.js_product-offer > div.product-offer__store-with-product > div.product-offer__store > div.product-offer__store__logo__rating > span.stars.js_mini-shop-info.js_no-conv > span.screen-reader-text')
        for i in range(len(temp_rating)):
            rating.append(temp_rating[i].text)
        rating = [w.replace('Ocena ', '').replace(' ','') for w in rating]

        # Product title
        temp_title = page.select('ul > li > div > div.product-offer__container.js_product-offer > div.product-offer__store-with-product > div.product-offer__product.js_product-offer__product.js_productName.specific-variant-content > div.product-offer__product__offer-details > div.product-offer__product__offer-details__name > a > span')
        for i in range(len(temp_title)):
            title.append(temp_title[i].text)

        # Number of ratings
        temp_rating_num = page.select('ul > li > div > div.product-offer__container.js_product-offer > div.product-offer__store-with-product > div.product-offer__store > div.product-offer__store__logo__rating > span.link.link--accent.js_mini-shop-info.js_no-conv')
        for i in range(len(temp_rating_num)):
            rating_number.append(temp_rating_num[i].text)
        rating_number = [w.replace(' opinie', '').replace(' opinii', '') for w in rating_number]
else:
    print("Access to " + tail + " denied")

## Create final data table and save to .csv file

In [9]:
data_table = pd.DataFrame(list(zip(
    store, rating, rating_number, price, title)),
    columns = ['Store_name','Positive_store_rating','Number_of_ratings','Price','Product_title']
)

In [11]:
data_table.to_csv('example_result.csv', sep='\t', encoding='utf-8')